In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

In [ ]:
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")

train_df.head()

In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

In [ ]:
from sklearn.model_selection import  train_test_split

In [ ]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size =0.1,  
    random_state =42
    )

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

In [ ]:
train_sentences[:5],train_labels[:5]

In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=10000, 
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None, 
    output_mode="int", 
    output_sequence_length=None, 
    pad_to_max_tokens=True
)

In [ ]:

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=10000,
    output_mode = "int",
    output_sequence_length=15
)

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
embedding = tf.keras.layers.Embedding(
    input_dim=10000, # set input shape
    embeddings_initializer="uniform",
    output_dim=128, # output shape
    input_length=15 # how long is each input
)

embedding

In [ ]:
model_1 = tf.keras.Sequential([
    tf.keras.Input(shape=(1,),dtype=tf.string),
    text_vectorizer,
    embedding,
    tf.keras.layers.Dense(32,activation="relu"),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32,activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model_1.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [ ]:
model_1_history = model_1.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences,val_labels)
)